In [1]:
import numpy as np
import pandas as pd

## Acquiring NeurIPS (2019) tweets:

In [2]:
%%bash
# snscrape --max-results 5000 twitter-hashtag neurips > neurips_tag_5000.txt
# snscrape --max-results 5000 twitter-hashtag neurips2020 > neurips2020_tag_5000.txt
# snscrape --max-results 5000 twitter-hashtag neurips2019 > neurips2019_tag_5000.txt
# snscrape --max-results 5000 twitter-hashtag neurips2018 > neurips2018_tag_5000.txt
# snscrape --max-results 5000 twitter-hashtag nips2018 > nips2018_tag_5000.txt
# snscrape --max-results 5000 twitter-hashtag nips2017 > nips2017_tag_5000.txt

# snscrape --max-results 5000 twitter-search nips > nips_search_5000.txt

In [3]:
# try nouns first
tweet_list = open("neurips_tag_scrape_1000.txt").readlines()

In [23]:
from collections import defaultdict

username_dict = defaultdict(int)
for line in tweet_list:
    username = line.replace("https://twitter.com/", "").split('/')[0]
    username_dict[username]+=1

user_df = pd.DataFrame([[k, v] for k, v in username_dict.items()], 
                       columns=['id', 'count'])


In [24]:
user_df.head(3)

,id,count
0,BorisAKnyazev,2
1,flareaudio,9
2,michellemng,1


In [6]:
user_df.sort_values("count", ascending=False).head(10)

,id,count
7,anoushnajarian,44
453,thinkmariya,29
9,AminerScholar,27
484,AISaturdaysIB,15
46,AnimaAnandkumar,12
489,Tijazz94,11
1,flareaudio,9
376,FmFrancoise,9
332,Abebab,9
331,AtticScientist,8


In [7]:
"""
for index, row in user_df.iterrows():
    !snscrape --max-results 200 twitter-user {row['id']}

! snscrape --max-results 5000 twitter-search "#neurips2019 since:2015-01-01 until:2020-03-15" > neurips2019_tweets.txt
    
"""

'\nfor index, row in user_df.iterrows():\n    !snscrape --max-results 200 twitter-user {row[\'id\']}\n\n! snscrape --max-results 5000 twitter-search "#neurips2019 since:2015-01-01 until:2020-03-15" > neurips2019_tweets.txt\n    \n'

In [8]:
tweet_url = pd.read_csv("neurips2019_tweets.txt", index_col= None, header = None, names = ["links"])
af = lambda x: x["links"].split("/")[-1]
tweet_url['id'] = tweet_url.apply(af, axis=1)
tweet_url.head()

,links,id
0,https://twitter.com/boredyannlecun/status/1238...,1238623609846935553
1,https://twitter.com/ionispharma/status/1310596...,1310596334613868545
2,https://twitter.com/AnalyticaGlobal/status/123...,1238497697746890756
3,https://twitter.com/Oracle/status/123849521419...,1238495214190432261
4,https://twitter.com/OracleBelux/status/1238433...,1238433480633171969


In [9]:
import tweepy

consumer_key = "FRVGuMgEuhJNQxcMUgLT3MMao" 
consumer_secret = "rFWAW4A4yys6jvPcGdBY4K5wNfFA9Iy2BIQMqr9yhS2I9tYxn8" 
access_token = "939281442621743104-MgHDnMelzUZHeh2Sui0nkJ1oTYfcWHy" 
access_token_secret = "sjQkY0dye7IPJB4MmkSRqatN49gA7n7bNpgdR7liUeLOf"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [10]:
ids = tweet_url['id'].tolist()
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": int(status.id),
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at}
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("neurips2019_tweets.csv", mode="a")

# for i in range(chunks):
#         batch = ids[i*50:(i+1)*50]
#         result = fetch_tw(batch)

### Getting those tweets into Pandas for analysis:

In [11]:
df_2019 = pd.read_csv("neurips2019_tweets.csv", index_col=0)
df_2019.head(3)

,date,screen_name,tweet,tweet_id
0.0,2020-02-20 21:46:33,JoinRobinly,"@josh_tobin_ , former researcher @OpenAI &amp;...",1.230609711864529e+18
1.0,2020-03-09 16:05:16,Oracle,What happens to accuracy when we try to increa...,1.2370468083392635e+18
2.0,2020-02-21 08:18:33,ShayneLibby,Oracle: How can you improve both privacy and a...,1.2307687603880796e+18


In [12]:
df_2019.groupby('screen_name').count().sort_values(by='tweet',ascending=False)

,date,tweet,tweet_id
screen_name,,,
bgalbraith,483,483,483
JFutoma,369,369,369
screen_name,299,299,299
RahelJhirad,237,237,237
MeetVancouver,201,201,201
...,...,...,...
akbirkhan,3,3,3
akashmehra,3,3,3
ajitndesai,3,3,3


In [13]:
import re, string
def cleanTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF')
    return tweet

# Need to stem and/or lemmatize?

In [14]:
df_2019['clean_tweet'] = df_2019['tweet'].apply(cleanTweet)
df_2019.head(3)

,date,screen_name,tweet,tweet_id,clean_tweet
0.0,2020-02-20 21:46:33,JoinRobinly,"@josh_tobin_ , former researcher @OpenAI &amp;...",1.230609711864529e+18,former researcher phd shared highlights from h...
1.0,2020-03-09 16:05:16,Oracle,What happens to accuracy when we try to increa...,1.2370468083392635e+18,what happens to accuracy when we try to increa...
2.0,2020-02-21 08:18:33,ShayneLibby,Oracle: How can you improve both privacy and a...,1.2307687603880796e+18,oracle how can you improve both privacy and ac...


In [15]:
df_2019 = df_2019.fillna(value='')
corpus = df_2019['clean_tweet']

In [16]:
for tweet in corpus.sample(10).values:
    print(tweet)
    print()

within a few clicks a user goes from searching for an innocuous subject activated charcoal to believing activated charcoal is good for you huge change in belief system as a result of google search rankings neurips2019

creative gans for generating poems lyrics and metaphors by and at neurips4creativity neurips2019 congrats to all presenters 

7 sat 4 15 pm west 121 122 on science meets engineering of deep learning panel with aparna lakshmiratan moderated but never in moderation by and finally have a great neurips2019  

16 years old boy went to do ritual and the police arrest him to read more about this kindly click here missworld magicshopinosaka neurips2019 nffc nigeria pakistan phcn powerrangers qatar2019 quotes realsociedadbarca religiousdiscriminationbill

heard at the end of a talk at neurips2019 “finally for those interested in neuroscience ”

delighted to incoming to neurips2019 i’ll be giving talks at deep rl at 1 30pm west exhibition hall c and ml w guarantees at 4 45pm west 

## LSA with TruncatedSVD:

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

stop_words = ENGLISH_STOP_WORDS.union(['neurips', 'neurips2019'])

vectorizer = CountVectorizer(stop_words=stop_words)
doc_word = vectorizer.fit_transform(corpus)
doc_word.shape

(15299, 11108)

### TF-IDF prep:

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF

In [19]:
tfidf = TfidfVectorizer(stop_words=stop_words)
tweet_word_matrix = tfidf.fit_transform(corpus)
print(tweet_word_matrix.shape)
vocab = tfidf.get_feature_names()

(15299, 11108)


In [20]:
# We have to convert `.toarray()` because the vectorizer returns a sparse matrix.
# For a big corpus, we would skip the dataframe and keep the output sparse.
dtm_lsa = pd.DataFrame(doc_word.toarray(), index=corpus, columns=vectorizer.get_feature_names()).head(10)

In [21]:
# Latent Semantic Analysis (LSA) is just another name for Singular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.01541733, 0.01494579])

In [22]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

,00,000,000mph,00pm,01,02,02476,03,04,04adr,...,研究の進め方,結合タスク自己教師あり学習や音楽からダンスへの生成プロセスに関する5つの論文を発表しました,詳しくはこちらをご覧ください,論文の書き方まで説明してある,详解图的机器学习趋势,長蛇の列に,雷锋网,音楽,齋藤さん,ﾎﾎｩ
component_1,0.019,0.005,0.0,0.004,0.001,0.003,0.001,0.001,0.003,0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0.000,0.000,0.0
component_2,-0.034,-0.004,-0.0,-0.007,-0.002,0.001,0.002,0.000,-0.001,-0.0,...,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,0.001,0.001,-0.0


In [138]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [139]:
display_topics(lsa, vectorizer.get_feature_names(), 5)


Topic  0
learning, workshop, ai, poster, deep

Topic  1
learning, deep, machine, bengio, yoshua


In [141]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = corpus,
             columns = ["component_1","component_2" ])
Vt

,component_1,component_2
clean_tweet,,
former researcher phd shared highlights from his neurips2019 paper geometry aware neural rendering in the exclusive interview with,0.23613,-0.06739
what happens to accuracy when we try to increase fairness in machinelearning a paper presented at neurips2019 has the answer,0.27252,-0.10808
oracle how can you improve both privacy and accuracy when training machinelearning models learn what differentially private federated learning is in this blog post based on a neurips2019 paper,0.99299,0.51753
today we’re joined by ilias diakonikolas faculty at the and author of the paper distribution independent pac learning of halfspaces with massart noise recipient of the neurips2019 outstanding paper award,1.01388,0.30867
shared his latest research in deeplearning theory at neurips2019 the work behind his acm doctoral dissertation honorablemention award insights on the future of ai research view,0.77206,-0.59349
...,...,...
thanks to av technology presenter who arent able to attend due to denies visas can present remotely with great audio visual quality thank you organizers for making this happen neurips2019,0.27069,-0.20475
erick tornero talking about a reinforcement learning approach to fly quadcopters with a faulted rotor neurips2019,0.64867,0.67233
ran out of he him pronoun stickers pretty accurate gender representation neurips2019,0.03625,0.01366


In [142]:
cosine_similarity(Vt)

array([[1.        , 0.99504758, 0.72590048, ..., 0.80306505, 0.98360874,
        0.98455378],
       [0.99504758, 1.        , 0.65393838, ..., 0.73985647, 0.99666086,
        0.99708101],
       [0.72590048, 0.65393838, 1.        , ..., 0.99279924, 0.5899808 ,
        0.59426643],
       ...,
       [0.80306505, 0.73985647, 0.99279924, ..., 1.        , 0.68245309,
        0.68633064],
       [0.98360874, 0.99666086, 0.5899808 , ..., 0.68245309, 1.        ,
        0.99998586],
       [0.98455378, 0.99708101, 0.59426643, ..., 0.68633064, 0.99998586,
        1.        ]])

In [143]:
cos_dtm = cosine_similarity(dtm_lsa).round(2)
cos_dtm

array([[1.  , 0.1 , 0.08, 0.13, 0.07, 0.  , 0.  , 0.08, 0.  , 0.  ],
       [0.1 , 1.  , 0.25, 0.15, 0.  , 0.  , 0.08, 0.26, 0.  , 0.  ],
       [0.08, 0.25, 1.  , 0.17, 0.  , 0.  , 0.06, 0.97, 0.  , 0.  ],
       [0.13, 0.15, 0.17, 1.  , 0.05, 0.  , 0.  , 0.17, 0.  , 0.  ],
       [0.07, 0.  , 0.  , 0.05, 1.  , 0.  , 0.11, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.08, 0.06, 0.  , 0.11, 0.  , 1.  , 0.06, 0.  , 0.  ],
       [0.08, 0.26, 0.97, 0.17, 0.  , 0.  , 0.06, 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 1.  ]])

In [144]:
cos_dtm[0][1]

0.1

In [145]:
cos_dtm[0][6]

0.0

### NMF

In [112]:
# Let's try 5 topics:
nmf = NMF(n_components=5)
nmf.fit(tweet_word_matrix)

NMF(n_components=5)

#### Tweet/Topic Matrix

In [113]:
tweet_topic_matrix = nmf.transform(tweet_word_matrix)
tweet_topic_matrix_df = pd.DataFrame(tweet_topic_matrix).add_prefix('topic_')

tweet_topic_matrix_df = pd.concat([tweet_topic_matrix_df.reset_index(drop=True), df_2019[['tweet', 'clean_tweet']].reset_index(drop=True)], axis=1)
tweet_topic_matrix_df.head()

(15299, 5)
(15299, 5)


,topic_0,topic_1,topic_2,topic_3,topic_4,tweet,clean_tweet
0,0.000011,0.011145,0.000000,0.001420,0.006048,"@josh_tobin_ , former researcher @OpenAI &amp;...",former researcher phd shared highlights from h...
1,0.000000,0.006707,0.000000,0.002365,0.086019,What happens to accuracy when we try to increa...,what happens to accuracy when we try to increa...
2,0.000000,0.029209,0.000000,0.000320,0.069486,Oracle: How can you improve both privacy and a...,oracle how can you improve both privacy and ac...
3,0.000042,0.024172,0.006175,0.003147,0.001461,"Today we’re joined by Ilias Diakonikolas, facu...",today we’re joined by ilias diakonikolas facul...
4,0.000031,0.017927,0.000000,0.000000,0.048573,@tengyuma @StanfordAILab @Stanford shared his ...,shared his latest research in deeplearning the...


#### Word/Topic Matrix

In [114]:
word_topic_matrix_df = pd.DataFrame(nmf.components_, columns=vocab).T.add_prefix('topic_')
word_topic_matrix_df.head()

,topic_0,topic_1,topic_2,topic_3,topic_4
00,0.0,0.035743,0.002021,0.088625,0.000000
000,0.0,0.014926,0.000000,0.000000,0.017033
000mph,0.0,0.000467,0.000000,0.000050,0.000000
00pm,0.0,0.005736,0.001326,0.022943,0.000000
01,0.0,0.001735,0.000000,0.006525,0.000000


## Topic Interpretation:

In [130]:
tweet_topic_matrix_df.sort_values(by='topic_4', ascending=False).head(10)

,topic_0,topic_1,topic_2,topic_3,topic_4,tweet,clean_tweet
9731,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/u6E...,neurips2019 machinelearning
8875,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/PzW...,neurips2019 machinelearning
11939,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/RRO...,neurips2019 machinelearning
4631,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/u6E...,neurips2019 machinelearning
2801,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/scm...,neurips2019 machinelearning
7575,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/sii...,neurips2019 machinelearning
8212,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/Opl...,neurips2019 machinelearning
13918,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/MW1...,neurips2019 machinelearning
1254,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/56S...,neurips2019 machinelearning
13001,0.0,0.0,0.0,0.0,0.315025,#NeurIPS2019 #MachineLearning https://t.co/scm...,neurips2019 machinelearning


Currently, the topics are as follows:

1. topic_0 = invalid tweet
2. topic_1 = Yoshua Bengio
3. topic_2 = RNAbench2bedside
4. topic_3 = poster session links
5. topic_4 = tweets or retweets to other tweets

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models, similarities, matutils

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [30]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", max_df = 0.50)

count_vectorizer.fit(corpus)

CountVectorizer(ngram_range=(1, 2), stop_words='english',
                token_pattern='\\b[a-z][a-z]+\\b')

In [32]:
doc_word = count_vectorizer.transform(corpus).transpose()
pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,15289,15290,15291,15292,15293,15294,15295,15296,15297,15298
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa woman,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaand fear,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
gs_corpus = matutils.Sparse2Corpus(doc_word)

In [35]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
len(id2word)

50350

In [36]:
lda = models.LdaModel(corpus=gs_corpus, num_topics=3, id2word=id2word, passes=5)

ROGRESS: pass 2, at document #6000/15299
2020-11-08 20:13:50,918 : INFO : merging changes from 2000 documents into a model of 15299 documents
2020-11-08 20:13:50,928 : INFO : topic #0 (0.333): 0.008*"poster" + 0.006*"learning" + 0.005*"neurips" + 0.005*"work" + 0.005*"workshop" + 0.004*"today" + 0.004*"ai" + 0.004*"come" + 0.003*"paper" + 0.003*"research"
2020-11-08 20:13:50,929 : INFO : topic #1 (0.333): 0.012*"learning" + 0.007*"talk" + 0.006*"ai" + 0.006*"workshop" + 0.004*"ml" + 0.004*"deep" + 0.004*"neurips" + 0.003*"deep learning" + 0.003*"machine" + 0.003*"bengio"
2020-11-08 20:13:50,931 : INFO : topic #2 (0.333): 0.007*"ai" + 0.004*"poster" + 0.004*"great" + 0.004*"vancouver" + 0.003*"work" + 0.003*"workshop" + 0.003*"paper" + 0.003*"tweet" + 0.003*"talk" + 0.003*"time"
2020-11-08 20:13:50,932 : INFO : topic diff=0.175528, rho=0.306433
2020-11-08 20:13:50,959 : INFO : PROGRESS: pass 2, at document #8000/15299
2020-11-08 20:13:51,493 : INFO : merging changes from 2000 documents 

In [40]:
lda.print_topics()

2020-11-08 20:16:28,128 : INFO : topic #0 (0.333): 0.009*"poster" + 0.006*"learning" + 0.005*"work" + 0.005*"workshop" + 0.005*"today" + 0.005*"neurips" + 0.004*"come" + 0.004*"ai" + 0.004*"hall" + 0.003*"paper"
2020-11-08 20:16:28,131 : INFO : topic #1 (0.333): 0.012*"learning" + 0.007*"talk" + 0.005*"workshop" + 0.005*"ai" + 0.004*"ml" + 0.004*"deep" + 0.004*"neurips" + 0.003*"deep learning" + 0.003*"machine" + 0.003*"bengio"
2020-11-08 20:16:28,133 : INFO : topic #2 (0.333): 0.006*"ai" + 0.004*"poster" + 0.004*"vancouver" + 0.004*"great" + 0.003*"work" + 0.003*"tweet" + 0.003*"workshop" + 0.003*"talk" + 0.003*"paper" + 0.002*"ml"


[(0,
  '0.009*"poster" + 0.006*"learning" + 0.005*"work" + 0.005*"workshop" + 0.005*"today" + 0.005*"neurips" + 0.004*"come" + 0.004*"ai" + 0.004*"hall" + 0.003*"paper"'),
 (1,
  '0.012*"learning" + 0.007*"talk" + 0.005*"workshop" + 0.005*"ai" + 0.004*"ml" + 0.004*"deep" + 0.004*"neurips" + 0.003*"deep learning" + 0.003*"machine" + 0.003*"bengio"'),
 (2,
  '0.006*"ai" + 0.004*"poster" + 0.004*"vancouver" + 0.004*"great" + 0.003*"work" + 0.003*"tweet" + 0.003*"workshop" + 0.003*"talk" + 0.003*"paper" + 0.002*"ml"')]

In [37]:
lda_corpus = lda[gs_corpus]
lda_corpus

In [38]:
lda_docs = [doc for doc in lda_corpus]

In [39]:
lda_docs[0:5]

[[(0, 0.01832504), (1, 0.01683113), (2, 0.9648438)],
 [(0, 0.01989959), (1, 0.95987207), (2, 0.02022829)],
 [(0, 0.97782177), (1, 0.011347768), (2, 0.010830506)],
 [(0, 0.9672344), (1, 0.010698692), (2, 0.022066943)],
 [(0, 0.011756039), (1, 0.011537025), (2, 0.9767069)]]

## Order of operations:

1. MDS
1. TSNE
1. Cosine similarity vs.
1. Euclidian similarity
1. NLTK > SpaCy (small OK, can try medium)
1. Define tokenize and stem functions, plus list of stop words.
1. Apply Tf-Idf transformation, and compute matrix of cosine-similarity distances.
1. Perform multidimensional scale (MDS) transformation.
1. Plot the result of MDS.
1. Compare with TSNE
1. ScatterText
1. Word2Vec
1. Apply K-Means clustering with pre-specified number of clusters, and visualize result with MDS.
1. Apply LDA transformation with pre-specified number of topics, and with or without topic seeding.
1. PyLDAVis
1. Visualize result of LDA transformation with MDS.